# Model Development

In [ ]:
import pandas as pd
import os


from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# read data back in from pickle file created with eda.ipynb

# Dynamically get the current working directory
current_dir = os.getcwd()
text_path = os.path.abspath(os.path.join(current_dir, '..', 'output','combined_data.pkl'))

# read data back in 
df_clean = pd.read_pickle(text_path)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier# Combine the list of words in 'clean_text' back into a string for TF-IDF

df_clean['clean_text_str'] = df_clean['clean_text'].apply(lambda x: ' '.join(x))

#Turning the labels into numbers
label_encoder = LabelEncoder()
df_clean['Category_Num'] = label_encoder.fit_transform(df_clean['Category'])
print(df_clean['Category'].unique())
print(df_clean['Category_Num'].unique())

# Split the data into features (X) and labels (y)
X = df_clean['clean_text_str']
y = df_clean['Category_Num']

print (X.shape)
print(y.shape)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Fit TF-IDF on the training data
tfidf = TfidfVectorizer(max_features=100, stop_words='english')
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [ ]:
# Define multiple models to test
models = {
    'Naive Bayes': MultinomialNB(),
    'Support Vector Classifier': SVC(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(n_estimators=100)
}


# Evaluate each model
for model_name, model in models.items():
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)
    
    print(f"\n=== {model_name} ===")
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Cross-validate the models
for model_name, model in models.items():
    scores = cross_val_score(model, X_train_tfidf, y_train, cv=5, scoring='accuracy')
    print(f"{model_name} Cross-Validation Accuracy: {scores.mean():.4f}")


In [ ]:


# X_train, X_test, y_train, y_test = train_test_split(subjects['question_text'], subjects['actual_subject'], 
#                                                     random_state = 0)

# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(X_train)

# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# clf = MultinomialNB().fit(X_train_tfidf, y_train)